In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [ ]:
# plotting styles
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10,6)

### Loading Cleaned Datasets

This code loads the cleaned solar datasets for **Benin**, **Sierra Leone**, and **Togo** into a dictionary `dfs`.  

- Each CSV is read into a DataFrame.
- The `Timestamp` column is converted to datetime (if present).
- A `country` column is added to identify the country after concatenation.
- The shapes of the datasets are printed to confirm successful loading.

In [ ]:
# load datasets
paths = {
    "Benin": "../data/cleaned_data_benin.csv",
    "SierraLeone": "../data/cleaned_data_sierraleone.csv",
    "Togo": "../data/cleaned_data_togo.csv"
}

dfs = {}
for country, path in paths.items():
    df = pd.read_csv(path)
    # ensure Timestamp is datetime if present
    if "Timestamp" in df.columns:
        df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    df["country"] = country  # add column to identify country after concatenation
    dfs[country] = df

# quick sanity
for k,v in dfs.items():
    print(k, v.shape)


### Combining Datasets Across Countries

The `pd.concat()` function merges the individual country DataFrames stored in the `dfs` dictionary into a single DataFrame `df_all`.

- `list(dfs.values())` extracts all DataFrames from the dictionary.
- `ignore_index=True` resets the index in the combined DataFrame so that it runs continuously from 0 to the total number of rows.
- `df_all.sample(10)` displays 10 random rows from the combined dataset to quickly inspect the merged data.

**Purpose:**  
This allows us to perform comparative analysis across countries in a single DataFrame while retaining the `country` identifier column.


In [ ]:
# Cell 3: concat for plotting & summary
df_all = pd.concat(list(dfs.values()), ignore_index=True)
df_all.sample(10)

### Boxplots of Solar Metrics by Country

This cell creates side-by-side boxplots for `GHI`, `DNI`, and `DHI` to compare their distributions across Benin, Sierra Leone, and Togo.

- **Loop over metrics:** The `metrics` list ensures that we create a separate plot for each solar radiation metric.
- **Check column existence:** The `if metric in df_all.columns` ensures the metric exists in the DataFrame.
- **Figure size:** `plt.figure(figsize=(8,5))` sets a consistent plot size.
- **Boxplot:**
  - `x="country"` → groups data by country.
  - `y=metric` → plots the metric values.
  - `hue="country"` → colors the boxes by country (useful for visual clarity).
  - `palette="coolwarm"` → specifies the color scheme.
- **Titles and labels:** Set for clarity.
- **`plt.tight_layout()`** → adjusts spacing so labels and titles fit neatly.
- **`plt.show()`** → displays the plot.

**Purpose:**  
Visualize the distribution, median, interquartile range, and potential outliers of solar metrics for each country. This helps quickly identify differences in solar potential and variability across countries.


In [ ]:
# Cell 4: boxplots for GHI, DNI, DHI
metrics = ["GHI", "DNI", "DHI"]
for metric in metrics:
    if metric in df_all.columns:
        plt.figure(figsize=(8,5))
        sns.boxplot(x="country", y=metric, data=df_all, hue = "country",palette="coolwarm")
        plt.title(f"Distribution of {metric} by Country")
        plt.ylabel(metric)
        plt.xlabel("Country")
        plt.tight_layout()
        plt.show()
    else:
        print(f"{metric} not found in data columns.")


### Summary Table of Solar Metrics by Country

This cell creates a table summarizing the solar metrics (`GHI`, `DNI`, `DHI`) for each country. It shows the **mean**, **median**, and **standard deviation**, providing a clear comparison of central tendency and variability across Benin, Sierra Leone, and Togo. The table helps to quickly identify which country has higher average solar radiation and how consistent the values are within each country.


In [ ]:
# Cell 5: summary table
metrics = ["GHI", "DNI", "DHI"]
summary = df_all.groupby("country")[metrics].agg(['mean','median','std']).round(3)
# flatten column multiindex
summary.columns = ["_".join(col).strip() for col in summary.columns.values]
summary = summary.reset_index()
summary


### Statistical Testing on GHI Across Countries

This cell performs statistical tests to determine whether the **Global Horizontal Irradiance (GHI)** values differ significantly across Benin, Sierra Leone, and Togo.

1. **Sample Preparation**  
   - Extract GHI values for each country and remove any missing values (`dropna()`).
   - Samples are limited to a maximum of 5000 observations for normality testing due to Shapiro-Wilk limitations.

2. **Normality Test (Shapiro-Wilk)**  
   - Checks if the GHI distribution in each country is approximately normal.  
   - If `p-value > 0.05`, data is considered normally distributed.
   - Here, the test could not confirm normality (or wasn't fully applicable due to large sample sizes).

3. **Homogeneity of Variances (Levene’s Test)**  
   - Tests whether the variance of GHI is equal across countries.
   - `stat=3777.8335`, `p=0` → significant result; variances are unequal.

4. **Choice of Test**  
   - Since either normality or equal variance assumptions are not met, the **non-parametric Kruskal-Wallis test** is used instead of ANOVA.  
   - Kruskal-Wallis test compares medians across groups.  
   - Output: `H=6925.2798, p=0` → very small p-value indicates **significant differences in GHI** among the three countries.

**Interpretation:**  
- The Kruskal-Wallis result shows that GHI distributions differ significantly between Benin, Sierra Leone, and Togo.  
- This justifies further post-hoc analysis or pairwise comparisons to identify which countries differ most.


In [30]:
# Cell 6: tests on GHI
metric = "GHI"
available_countries = [c for c in paths.keys() if metric in dfs[c].columns]

# prepare samples
samples = [dfs[c][metric].dropna().values for c in available_countries]

# 1) normality test (Shapiro) - note: Shapiro may be sensitive for large n; interpret carefully
norm_results = {}
for c in available_countries:
    vals = dfs[c][metric].dropna().sample(n=min(5000, dfs[c][metric].dropna().shape[0]), random_state=0)  # sample to reasonable size
    stat, p = stats.shapiro(vals) if len(vals) <= 5000 else (np.nan, np.nan)  # Shapiro requires n <= 5000
    norm_results[c] = {"W": stat, "pvalue": p}
pd.DataFrame(norm_results).T

# 2) homogeneity of variances (Levene)
levene_stat, levene_p = stats.levene(*samples, center='median')
print(f"Levene test: stat={levene_stat:.4f}, p={levene_p:.4g}")

# 3) choose test
if (all((v["pvalue"] is not np.nan and v["pvalue"] > 0.05) for v in norm_results.values()) 
    and levene_p > 0.05):
    print("Data approximately normal and variances equal → run one-way ANOVA")
    f_stat, p_anova = stats.f_oneway(*samples)
    print(f"ANOVA result: F={f_stat:.4f}, p={p_anova:.4g}")
else:
    print("Normality/variance assumptions not met → run Kruskal-Wallis")
    h_stat, p_kw = stats.kruskal(*samples)
    print(f"Kruskal-Wallis result: H={h_stat:.4f}, p={p_kw:.4g}")


Levene test: stat=3777.8335, p=0
Normality/variance assumptions not met → run Kruskal-Wallis
Kruskal-Wallis result: H=6925.2798, p=0


### Post-hoc Analysis (Tukey HSD) on GHI

This cell performs **Tukey's Honestly Significant Difference (HSD)** test as a post-hoc analysis to determine **which pairs of countries differ significantly** in their GHI values.  

1. **Purpose:**  
   - After finding a significant difference using ANOVA (or Kruskal-Wallis), we use Tukey HSD to identify **specific country pairs** where the difference is statistically significant.  
   - Compares **all possible pairs** while controlling the family-wise error rate (FWER).

2. **Code Explanation:**  
   - `endog=df_all[metric].dropna()`: The dependent variable (GHI values).  
   - `groups=df_all.loc[df_all[metric].notna(), "country"]`: The categorical group variable (countries).  
   - `alpha=0.05`: Significance level; p-values below 0.05 indicate significant differences.

3. **Output Interpretation:**  
   | group1 | group2 | meandiff | p-adj | lower | upper | reject |
   |--------|--------|----------|-------|-------|-------|--------|
   | Benin  | SierraLeone | -52.38 | 0.0   | -53.80 | -50.95 | True |
   | Benin  | Togo        | -10.58 | 0.0   | -12.00 | -9.16  | True |
   | SierraLeone | Togo   | 41.80  | 0.0   | 40.37  | 43.23  | True |

   - `meandiff`: Difference in mean GHI between the two countries.  
   - `p-adj`: Adjusted p-value accounting for multiple comparisons.  
   - `reject=True`: Null hypothesis (means equal) is rejected; difference is **statistically significant**.  

4. **Key Observations:**  
   - GHI in **Benin is significantly lower than in Sierra Leone** by ~52 W/m².  
   - GHI in **Benin is also lower than Togo** by ~11 W/m².  
   - **Sierra Leone has significantly higher GHI than Togo** by ~42 W/m².  
   - All differences are statistically significant at 5% significance level.


In [31]:
# Cell 7: post-hoc (Tukey) if ANOVA used
try:
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    if metric in df_all.columns:
        tukey = pairwise_tukeyhsd(endog=df_all[metric].dropna(), groups=df_all.loc[df_all[metric].notna(), "country"], alpha=0.05)
        print(tukey.summary())
except Exception as e:
    print("Tukey test unavailable (statsmodels not installed) or error:", e)


      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1      group2   meandiff p-adj  lower    upper   reject
---------------------------------------------------------------
      Benin SierraLeone -52.3755   0.0 -53.8025 -50.9486   True
      Benin        Togo -10.5763   0.0 -11.9956  -9.1571   True
SierraLeone        Togo  41.7992   0.0  40.3706  43.2278   True
---------------------------------------------------------------


In [ ]:
# Cell 8: bar chart of mean GHI
if "GHI" in df_all.columns:
    mean_ghi = df_all.groupby("country")["GHI"].mean().sort_values(ascending=False)
    ax = mean_ghi.plot(kind="bar", color=["#4c72b0","#55a868","#c44e52"])
    plt.ylabel("Average GHI")
    plt.title("Average GHI by Country (ranked)")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
